# Custom Loss Functions and Predictors


# Loading packages

In [2]:
import torch
from torch import Tensor
from torch.nn import functional as F

import pandas as pd
from pathlib import Path

from chemprop.nn.loss import LossFunction
from chemprop.nn.metrics import Metric
from chemprop import data, featurizers, models, nn

from chemprop.nn.predictors import RegressionFFN

In [3]:
print(nn.LossFunctionRegistry)

ClassRegistry {
    'mse': <class 'chemprop.nn.loss.MSELoss'>,
    'bounded-mse': <class 'chemprop.nn.loss.BoundedMSELoss'>,
    'mve': <class 'chemprop.nn.loss.MVELoss'>,
    'evidential': <class 'chemprop.nn.loss.EvidentialLoss'>,
    'bce': <class 'chemprop.nn.loss.BCELoss'>,
    'ce': <class 'chemprop.nn.loss.CrossEntropyLoss'>,
    'binary-mcc': <class 'chemprop.nn.loss.BinaryMCCLoss'>,
    'multiclass-mcc': <class 'chemprop.nn.loss.MulticlassMCCLoss'>,
    'binary-dirichlet': <class 'chemprop.nn.loss.BinaryDirichletLoss'>,
    'multiclass-dirichlet': <class 'chemprop.nn.loss.MulticlassDirichletLoss'>,
    'sid': <class 'chemprop.nn.loss.SIDLoss'>,
    'earthmovers': <class 'chemprop.nn.loss.WassersteinLoss'>,
    'wasserstein': <class 'chemprop.nn.loss.WassersteinLoss'>
}


In [ ]:
class MSLELoss(LossFunction):
    def __init__(self, task_weights=None):
        super().__init__(task_weights)

    def _calc_unreduced_loss(self, preds, targets, mask, weights, lt_mask, gt_mask) -> Tensor:
        return (torch.log(preds + 1) - torch.log(targets + 1)) ** 2

In [ ]:
class HuberLoss(LossFunction):
    def __init__(self, task_weights=None, delta: float = 1.0):
        super().__init__(task_weights)
        self.register_buffer("delta", delta)

    def _calc_unreduced_loss(self, preds : Tensor, targets : Tensor, *args):
        return F.huber_loss(preds, targets, reduction="none", delta=self.delta)